**Connecting drive to this project and creating chunks from pdf**

In [1]:
!pip -q install langchain tiktoken chromadb pypdf sentence-transformers==2.2.2 InstructorEmbedding faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 7.8 MB/

In [2]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader

In [3]:
# InstructorEmbedding
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [4]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive"

Mounted at /content/gdrive


In [5]:
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader(f'{root_dir}/Documents/', glob="./*.pdf", loader_cls=PyPDFLoader)
documents = loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
                                               chunk_size=1000,
                                               chunk_overlap=200)

texts = text_splitter.split_documents(documents)

In [7]:
texts[30]

Document(page_content='Canova. For espresso coffee-lovers, we \nrecommend Caffè Sant’Eustachio located in \nthe square with the same name, a fine coffee \nshop founded in the 1930s where the coffee is \nroasted by hand over wood fires.Simple, with strong flavors, overflowing \nwith dressings and anything but low-calorie. \nPerhaps a little unrefined but this contributes \nto its authentic, genuine style. Traditional \nRoman cooking is made up of simple, meager \ningredients, that follow the seasons and which \nare therefore extremely fresh.\nA typical Roman menu begins with the \nessential bruschetta  “ammazzavampiri” \n(so full of garlic it would kill vampires), \nand maybe also a wonderful mozzarella in \ncarrozza . The large pasta course that follows \ncould be: spaghetti alla carbonara , bucatini \nall’amatriciana , bucatini cacio e pepe or \ngnocchi alla romana . If you want to try \nrigatoni pasta with the famous pajata sauce, \nwe recommend that first you eat it and then \nask w

In [8]:
len(texts)

73

***Creating embeddings from pdf***

In [9]:
!pip install faiss-cpu

In [10]:
import pickle
import faiss
from langchain.vectorstores import FAISS

In [11]:
### This function store created embeddings ###
def store_embeddings(docs, embeddings, sotre_name, path):

    vectorStore = FAISS.from_documents(docs, embeddings)

    with open(f"{path}/faiss_{sotre_name}.pkl", "wb") as f:
        pickle.dump(vectorStore, f)

In [12]:
### This function loads created embeddings ###
def load_embeddings(sotre_name, path):
    with open(f"{path}/faiss_{sotre_name}.pkl", "rb") as f:
        VectorStore = pickle.load(f)
    return VectorStore

In [13]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.40k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [19]:
Embedding_store_path = f"{root_dir}/Embedding_store"

In [20]:
db_instructEmbedd = FAISS.from_documents(texts, instructor_embeddings)

In [21]:
retriever = db_instructEmbedd.as_retriever(search_kwargs={"k": 3})

In [22]:
retriever.search_type

'similarity'

In [23]:
retriever.search_kwargs

{'k': 3}

In [24]:
docs = retriever.get_relevant_documents("Where can i get best espresso coffee?")

In [25]:
docs[0]

Document(page_content='Canova. For espresso coffee-lovers, we \nrecommend Caffè Sant’Eustachio located in \nthe square with the same name, a fine coffee \nshop founded in the 1930s where the coffee is \nroasted by hand over wood fires.Simple, with strong flavors, overflowing \nwith dressings and anything but low-calorie. \nPerhaps a little unrefined but this contributes \nto its authentic, genuine style. Traditional \nRoman cooking is made up of simple, meager \ningredients, that follow the seasons and which \nare therefore extremely fresh.\nA typical Roman menu begins with the \nessential bruschetta  “ammazzavampiri” \n(so full of garlic it would kill vampires), \nand maybe also a wonderful mozzarella in \ncarrozza . The large pasta course that follows \ncould be: spaghetti alla carbonara , bucatini \nall’amatriciana , bucatini cacio e pepe or \ngnocchi alla romana . If you want to try \nrigatoni pasta with the famous pajata sauce, \nwe recommend that first you eat it and then \nask w

In [26]:
docs[1]

Document(page_content='are also culprits in stirring up this desire for \nfood.\nFor good wine lovers, many wine cellars  \nin the city organize wine tasting courses \nand information-giving meetings about \nvineyards, harvesting, fermentation and all \nthe processes linked to the production of this \nnectar of the gods. Among some of the places \nin the city that organize such events, we can \nname “La Tradizione” and “Franchi”.\nThe historical coffee shops  in the city are \nour last stop in this section, where it is \npossible to try typical Roman food in unique \nsurroundings that combine culture, history \nand tradition. Meeting places and places \nto swap ideas for artists and writers in the \n19th and 20th centuries such as Caffè Greco, \nBabington’s Tearooms, Caffè Rosati and Caffè \nCanova. For espresso coffee-lovers, we \nrecommend Caffè Sant’Eustachio located in \nthe square with the same name, a fine coffee \nshop founded in the 1930s where the coffee is', metadata={'source

In [27]:
docs[2]

Document(page_content='and 60s. There are no longer actors or exiled \nmonarchs sitting at the coffee shop tables, \nbesieged by paparazzi. You may perhaps still \nfind a few out-of-date tourists.\nThe spirit of “la Dolce Vita” has moved with \nthe times, no longer aimed at the bored, \ndecadent aristocracy but to a large group of \nmildly unconventional night people that want \nto have a good time, communicate and relax.\nThis is a self-gratifying and slightly \nirresponsible attitude that has invaded the \nnarrow streets of Trastevere . Trastevere is \nan extremely lively area of town: walking \nalong the labyrinth of alleys and lanes, among \ncoffee shops, restaurants, pubs and clubs with \nlive music, all brimming over with life is a \nunique and exciting experience. \nThe areas around Via della Pace  and Campo \nde’ Fiori  are also interesting for night life La Dolce Vita\nRome : Bar Settimiano', metadata={'source': '/content/gdrive/My Drive/Documents/rome_guide.pdf', 'page': 10})